In [23]:
from pathlib import Path
# モデルのディレクトリ
# strで指定すると、from_pretrainedでなぜかHuggingFaceを参照したため、PathLikeで指定した。
output = Path("/home/n.yumeka/workspace/wrime/gakusyu/output_arinasi1") 
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-small")
tokenizer.do_lower_case = True
model = AutoModelForCausalLM.from_pretrained(output)
model.to(device)
model.eval()  


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [24]:

def generate_reply(inp, num_gen=1):
    input_text = "<s>" + str(inp) + "[SEP]"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    # 生成時のパラメータ変更
    out = model.generate(input_ids, do_sample=True, max_length=30, num_return_sequences=num_gen, top_p=0.90, top_k=10, bad_words_ids=[[1], [5]], no_repeat_ngram_size=1)

    #print(">", "あなた")
    #print(inp)
    #print(">", "システム")
    for sent in tokenizer.batch_decode(out):
        sent = sent.split('[SEP]</s>')[1]
        sent = sent.replace('<unk>', '')
        sent = sent.replace('</s>', '')
        sent = sent.strip('w')
        #print(sent)
        return sent

In [3]:
sentence = ["台湾もねー、遊びに行きたいけど。","そんだけかけてきて、ここに1時間半しか授業聞かないのってなんかもったいない気がするでしょ。。", "あーあ。最近ほんと冷蔵庫何も入ってないんだよ。", "はや1週間たっちゃったーっと思って", "すごい恐怖だって。", "手入れしてるひげだったらいいのかなあ", "私は日本の文化には合わないと思う。", "ひげ伸ばそうかな。", "私は日本の文化がすきだよ", "仕事が忙しくて寝る暇もない。たまには休ませてほしい…", "ろくに義務も果たしていないのに、権利ばかり主張して困っているんだよ", "今日は花粉がすごくてくしゃみが止まらない", "今年は帰省するか迷うな～"]
sentence1 = ["最近物価が高すぎて、卵が300円もする","皆に運転荒いっていわれる","勉強したいけど中々時間がとれない","小旅行感覚で友達と計画してるんだ","焼き肉が食べたかった。次は食べれるといいな","この曲すっごい懐かしいよね","歩きスマホやめてほしい","人間関係で悩んでるんだよね"]

In [53]:
msg = "この年になって、遅ればせながらピアノを習い始めました！"
generate_reply(msg)











The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'ねー。なんかもうちょっと早く教えてくれればよかったのに'

In [10]:
# 一行ずつファイルを読み込み、一部分を取り出す
import re
with open("../data/arinasi/arinasi1_test.txt", 'r') as file:
    text = file.read()
    # かっこを全て削除
    text = re.sub(r'，', ' 、', text)
    text = re.sub(r'<s>', ' ', text)
    text = re.sub(r'\[SEP\]', ' ', text)
    text = re.sub(r'</s>', ' ', text)

    #print(text)

with open('arinasi_kakou1.txt', 'w', encoding='utf-8') as f:
    f.write(text)  
        
        

In [190]:
import pandas as pd
kaiwa = pd.read_table('arinasi_kakou1.txt', sep = ' ')
kaiwa.columns = ['a', '発話', '応答','b']
# kaiwa
hatuwa = kaiwa["発話"]

hatuwa_list = hatuwa.tolist()
kaiwa


,a,発話,応答,b
0,NaN,わたしは知らないけどー、人がそう言うじゃん。,うん、そうだね。,NaN
1,NaN,フィットは生産が追いついてないみたい。,あ、そうなの。そんな人気あるんだねー。,NaN
2,NaN,そうそうそうそう。えっ、この人たちは小学生じゃないけど、ほら見て、こういうオーディションがあ...,知ってる子いないかな。,NaN
3,NaN,ふーん。,そう、だからー、一度お姉ちゃんのおうちへ行ってー、チーズを下ろして、ほんで来たの。,NaN
4,NaN,会話の録音なんてちょっとドキドキするんです。,あたしもちょっとドキドキです。,NaN
...,...,...,...,...
480,NaN,日本人って足短いんだもーん。,うーん、そうだねー。,NaN
481,NaN,日本の学生は質問しなさいっていう風に習ってないからね。,うーん。質問したら悪いと思ってるんですね、きっと。でもまじめなんですよね。,NaN
482,NaN,で、材木はあるけど、もうつか、使わないっていうから、お願いしてよかった。,どういうふうに?,NaN
483,NaN,反対に。,怖いかもしれん。教習車が的だね、一番。,NaN


In [8]:




import pandas as pd
#df.to_csv('kyoukanrei.csv', index = False)
msgg = []
anss = []

for f in range(len(hatuwa_list)):
    msg =[]
    msg =[hatuwa_list[f]]
    msgg.append(msg)
    ans = []
    #print(f)
    for i in range(len(msg)):

        ans.append(generate_reply(msg[i]))
        
    anss.append(ans)
    
flat_list1 = [item for sublist in msgg for item in sublist]  
flat_list2 = [item for sublist in anss for item in sublist]

d = pd.DataFrame(flat_list1, columns = ["ユーザ"])
df = pd.DataFrame(flat_list2, columns=["システム"])
df = pd.concat([d, df], axis = 1)

df


#with open('kyoukanrei.csv', mode = 'a', encoding='utf-8') as f:
    #f.write(df)
# df.to_csv("filtering_arinashi.csv", index = False, encoding = "UTF-8")
#df.to_csv('kyoukanrei.csv', index = False)
#df.to_csv("kyoukanrei.csv", mode='a', index = False, encoding = "UTF-8")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

,ユーザ,システム
0,わたしは知らないけどー、人がそう言うじゃん。,うん?なんかそういうのがあるっていう感じがしていいね
1,フィットは生産が追いついてないみたい。,うん、そうなんだーって言ってもらえて嬉しいねえ2ちゃん
2,そうそうそうそう。えっ、この人たちは小学生じゃないけど、ほら見て、こういうオーディションがあ...,そ
3,ふーん。,うん、それはそうかもなあと思ったよねえとかね?そしたらまたなんかさり
4,会話の録音なんてちょっとドキドキするんです。,違う、これって?おもしろかったなーと思ってねえ
...,...,...
480,日本人って足短いんだもーん。,あ、そうなんですか?なんかねえね...これでよかったのかなと思って
481,日本の学生は質問しなさいっていう風に習ってないからね。,うん、そうなんだよーとかいうかなんかね
482,で、材木はあるけど、もうつか、使わないっていうから、お願いしてよかった。,うんねーとかいっちゃ
483,反対に。,うん、そうだよねーって言われるよなあと思ってさ、「あの人どうしちゃったのかな


In [10]:
import pandas as pd
df.to_excel("filtering_arinashi.xlsx",index = False, encoding = "UTF-8")


In [2]:
import pandas as pd
df = pd.read_csv("filtering_arinashi.csv")
df

,ユーザ,システム
0,最近物価が高すぎて、卵が300円もする,前と変わってない?あれは。前のをもっと買うけど
1,最近物価が高すぎて、卵が300円もする,ん?なんかよくわからんけど。500とかのや
2,最近物価が高すぎて、卵が300円もする,じゃあ。うちでも500〜600個ぐらいあるけどさー
3,最近物価が高すぎて、卵が300円もする,いいじゃん。でもねーこれさあ1個250?
4,最近物価が高すぎて、卵が300円もする,はい。私ねー10分1万で買った
...,...,...
235,人間関係で悩んでるんだよね,。コミュニケーションって難しいことなの?私、これなんかちょっとだけどもめ
236,人間関係で悩んでるんだよね,。今の時期は、なんかよくない?omenonnoneとかいうやつ
237,人間関係で悩んでるんだよね,私、もう2年半も前なんですが。あのー私は何ていうの
238,人間関係で悩んでるんだよね,だって、私も仕事してて。なんか嫌なことあった?うんその


In [3]:
df.to_excel("filtering_nashi.xlsx", index = False, encoding = "UTF-8")

In [61]:
nested_list = [[1, 2], [3, 4], [5, 6]]
flat_list = [item for sublist in nested_list for item in sublist]
flat_list

[1, 2, 3, 4, 5, 6]

In [11]:
k = []
k = ["少し変えて訓練(元)"]
kf = pd.DataFrame(k)
kf.to_csv("kyoukanrei5.csv", mode='a', index = False, encoding = "UTF-8", header = False)


In [19]:
msgg =[]
msgg =["今年は帰省するか迷うな～"] *30

anss = []
for i in msgg:

    anss.append(generate_reply(i))
#d2 = pd.DataFrame(k)
d1 = pd.DataFrame(msgg, columns = ["ユーザ"])
df2 = pd.DataFrame(anss, columns=["システム"])
df2 = pd.concat([d1, df2], axis = 1)
df2

df2.to_csv("kyoukanrei5.csv", mode='a', index = False, encoding = "UTF-8", header = False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

In [67]:
import pandas as pd
msg =[]
msg =["私は日本の文化がすきだよ"] *5
ans = []
for i in msg:
     
    ans.append(generate_reply(i))

df = pd.DataFrame(ans, columns=['システム'])

df

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

,システム
0,うん、うん。
1,ほんと、日本の文化が好きだよね。
2,そうそう、わかる、わかる。
3,うん、そうね。
4,はいはい、わかりました


In [65]:
lsit = [1,2,3,4]
llll = ["今日も眠いな"]

dfg = pd.DataFrame(lsit, columns=['システム'])
dfg = df.append(llll, index=0)  

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_32923/487163493.py:5 in <cell line: 5>                                     │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_32923/487163493.py'                  │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: append() got an unexpected keyword argument 'index'

In [8]:
import pandas as pd
msg =[]
msg =["ろくに義務も果たしていないのに、権利ばかり主張して困っているんだよ"] *5
ans = []
for i in msg:
     
    ans.append(generate_reply(i))

df = pd.DataFrame(ans, columns=['システム'])

df

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

,システム
0,そうそう。
1,うん。
2,そうそうそう、義務も働いていないのにその権利ばかりを主張して困っているんですよね。
3,そうそうそう、権利はねえ。
4,うん。


In [10]:
import pandas as pd
msg =[]
msg =["今日は花粉がすごくてくしゃみが止まらない"] *5
ans = []
for i in msg:
     
    ans.append(generate_reply(i))

df = pd.DataFrame(ans, columns=['システム'])

df

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

,システム
0,花粉症の
1,さん
2,そらそらー そら、そら。
3,あ、花粉のね。
4,花粉症なんです。私もなんですけど、最近鼻づまりがすこしはあるんですよね。でも今年はどうなんで...


In [12]:
import pandas as pd
msg =[]
msg =["今年は帰省するか迷うな～"] *5
ans = []
for i in msg:
     
    ans.append(generate_reply(i))

df = pd.DataFrame(ans, columns=['システム'])

df

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

,システム
0,そうそうそう。
1,そりゃそうでしょう。
2,そうそうそう、帰省するよね。
3,いやいや、それはもうさ。
4,そうなんですよ。。( ́ー)ノ
